# Модель экстренных вызовов

В этой тетради мы проведем подбор признаков. Проведем ручную кластеризацию по двум категориям - ложный вызов (класс 0), авария (класс 1).

Анализ будет производится по сигналам из таблицы revshift_20-2.csv. Мы возмем из нее пути до аудио файлов.

In [2]:
import librosa # параметризация аудио
import numpy as np # математические формулы
import pandas as pd

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Таблица с путями до аудио сигналов
data = pd.read_csv('revshift_20.csv').drop_duplicates('label')


Создадим функцию с первичным набором признаков аудио сигнала, что бы определить лучшую функция обобщения массива данных и функция вывода результата на экран для поиска закономерностей по классам.

Наша цель найти признаки, где разница между двумя классами будет макисмальной.


In [6]:
#@title Функция параметризации аудио с определением метода обощения
def get_features(y, sr, f=np.mean):
  #Получаем различные параметры аудио
  chroma_stft = list(map(f, librosa.feature.chroma_stft(y=y, sr=sr))) # Частота цветности (по умолчанию 12 баков цветности)

  rmse = f(librosa.feature.rms(y=y)) #Среднеквадратичная амплитуда
  spec_cent = f(librosa.feature.spectral_centroid(y=y, sr=sr)) #среднее спектральныго центроида
  spec_bw = f(librosa.feature.spectral_bandwidth(y=y, sr=sr)) #среднее ширины полосы частот
  rolloff = f(librosa.feature.spectral_rolloff(y=y, sr=sr)) #среднее спектрального спада частоты
  zcr = f(librosa.feature.zero_crossing_rate(y)) #среднее частота пересечения нуля звукового временного ряда

  #Добавляем все параметры в один список
  out = [] # создаем пустой список
  out.append(rmse) #  добавляем среднеквадратичную амплитуду
  out.append(spec_cent) #добавляем спектральный центроид
  out.append(spec_bw) # добавляем ширину полосы частот
  out.append(rolloff) # добавляем спектральный спад частоты
  out.append(zcr) # добавляем пересечение нуля
  out.append(f(chroma_stft)) # добавляем
  return out

In [13]:
#@title Функция вывода результата на экран для поиска закономерностей по классам
def features_info(index_list, f):
    print('Таблица распределений', index_list)
    index_list = data.query(index_list).index
    lst = []
    for i in index_list:
        y, sr = librosa.load(data.loc[i,'original_path'])
        lst.append(get_features(y, sr, f))
    lst = pd.DataFrame(lst)
    display(lst.describe())
    print()

In [15]:
%%time
#@title Признаки с методом np.min()
f = np.min #@param
features_info('cls == 0', f=f)
features_info('cls == 1', f=f)

Таблица распределений cls == 0


,0,1,2,3,4,5
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.000408,564.337291,468.680150,845.716553,0.027230,0.005862
std,0.000705,221.111705,174.767781,556.390299,0.013647,0.018003
min,0.000003,3.684752,17.462004,10.766602,0.000000,0.000040
25%,0.000007,414.339442,345.804166,406.439209,0.018555,0.000117
50%,0.000015,492.887775,387.013352,592.163086,0.022461,0.000284
75%,0.000491,747.616293,667.712337,1356.591797,0.037109,0.002597
max,0.003001,1259.035115,997.874798,2702.416992,0.069336,0.128443



Таблица распределений cls == 1


,0,1,2,3,4,5
count,4.030000e+02,403.000000,403.000000,403.000000,403.000000,403.000000
mean,1.615577e-04,388.724973,359.414644,390.162405,0.017605,0.000231
std,2.506953e-04,175.521037,114.153666,217.747088,0.010845,0.002728
min,3.113112e-07,3.685809,17.674359,10.766602,0.000000,0.000004
25%,1.357068e-06,306.362699,343.522049,312.231445,0.010742,0.000010
50%,3.474475e-05,429.238711,382.604924,462.963867,0.018066,0.000014
75%,2.831662e-04,513.955764,407.729861,484.497070,0.024414,0.000023
max,2.526199e-03,1153.743350,768.584335,2164.086914,0.054199,0.054367



CPU times: total: 5min 59s
Wall time: 4min 21s


Вывод. Обнаружена значительная разница в признаках: 0, 1, 2, 3, 4, 5, 6

In [18]:
#@title Признаки с методом np.mean()
f = np.mean #@param
features_info('cls == 0', f=f)
features_info('cls == 1', f=f)

Таблица распределений cls == 0


,0,1,2,3,4,5
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.110517,1212.664105,870.435407,2200.053423,0.091796,0.424157
std,0.107028,222.264895,92.382396,334.361716,0.026518,0.110869
min,0.000288,834.354502,661.183051,1492.089265,0.042949,0.287169
25%,0.033695,1051.726850,818.921520,1981.315433,0.072092,0.339858
50%,0.068756,1199.188064,864.694706,2186.495674,0.087602,0.383935
75%,0.158417,1308.999822,902.945428,2344.927104,0.108055,0.520521
max,0.392343,1733.518584,1192.936277,3129.588894,0.155742,0.762237



Таблица распределений cls == 1


,0,1,2,3,4,5
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.059316,1197.613302,859.784663,2125.219150,0.091849,0.412786
std,0.042651,192.710159,91.635740,332.714619,0.022563,0.075505
min,0.004644,460.893459,580.619676,742.725253,0.025182,0.203164
25%,0.033359,1100.651904,802.929583,1915.381655,0.079620,0.363605
50%,0.048702,1200.031405,852.146701,2142.273914,0.090899,0.402314
75%,0.074189,1319.784999,911.580092,2314.007746,0.105106,0.457647
max,0.407895,2099.379239,1160.540533,3137.459197,0.219332,0.756308


Вывод. Обнаружена значительная разница в признаке: 0

In [19]:
#@title Признаки с методом np.max()
f = np.max #@param
features_info('cls == 0', f=f)
features_info('cls == 1', f=f)

Таблица распределений cls == 0


,0,1,2,3,4,5
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.0
mean,0.360295,2486.202508,1273.871042,3383.942871,0.240983,1.0
std,0.205364,453.906650,105.428493,211.374988,0.053812,0.0
min,0.000330,1187.459656,1045.892418,2680.883789,0.088867,1.0
25%,0.229197,2193.464871,1224.380534,3273.046875,0.214355,1.0
50%,0.339320,2577.050167,1271.967408,3445.312500,0.257324,1.0
75%,0.493350,2849.566069,1343.321901,3531.445312,0.285400,1.0
max,0.767691,3162.646603,1550.788305,3714.477539,0.307129,1.0



Таблица распределений cls == 1


,0,1,2,3,4,5
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.0
mean,0.372304,2753.503496,1329.288253,3505.370367,0.274929,1.0
std,0.151895,318.150075,88.677900,167.180802,0.033088,0.0
min,0.015468,1146.952385,933.738730,1970.288086,0.090820,1.0
25%,0.244092,2598.871864,1277.161371,3466.845703,0.263672,1.0
50%,0.323269,2806.106323,1330.145192,3531.445312,0.281250,1.0
75%,0.470254,2968.765166,1388.559459,3585.278320,0.294922,1.0
max,0.802991,3372.312191,1621.866560,3768.310547,0.335449,1.0


Вывод. Обнаружена значительная разница в признаках: 1, 2, 3

In [20]:
#@title Признаки с методом np.median()
f = np.median #@param
features_info('cls == 0', f=f)
features_info('cls == 1', f=f)

Таблица распределений cls == 0


,0,1,2,3,4,5
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.091825,1193.425641,886.053372,2262.691040,0.086609,0.350604
std,0.111565,240.862725,104.651981,389.069303,0.029898,0.166569
min,0.000270,798.725897,645.040831,1324.291992,0.024902,0.146422
25%,0.014795,1011.895649,818.645765,1989.129639,0.063965,0.221107
50%,0.047375,1159.846018,871.917541,2250.219727,0.080078,0.298187
75%,0.112213,1314.377440,921.770753,2532.843018,0.103882,0.477356
max,0.390436,1736.637491,1204.164567,3154.614258,0.157715,0.915134



Таблица распределений cls == 1


,0,1,2,3,4,5
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.027073,1210.333900,890.490451,2297.133256,0.087687,0.342920
std,0.040819,221.952007,117.032755,417.219606,0.025941,0.132062
min,0.000252,405.501010,570.772897,624.462891,0.009277,0.018681
25%,0.004631,1092.841201,816.964883,2067.187500,0.071777,0.246589
50%,0.016248,1215.961218,879.091971,2304.052734,0.087402,0.330123
75%,0.034177,1336.375087,953.378162,2594.750977,0.102051,0.423777
max,0.471528,2161.547290,1276.062346,3359.179688,0.229492,0.871485


Вывод. Обнаружена значительная разница в признаках: 0

# Новый набор признаков с индивидуальными методами обощения
С учетом результатов, пишем новую функцию параметризации аудио с новым набором параметров и их индивидульаными методами обощениями.

In [ ]:
%%time
#@title Новая функция параметризации аудио с индивидуальными методами обощения 
def get_features(y, sr):

    #Получаем различные параметры аудио
    rmse = librosa.feature.rms(y=y) # Среднеквадратичная амплитуда
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr) #среднее спектральныго центроида
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr) #среднее ширины полосы частот
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr) #среднее спектрального спада частоты
    zcr = librosa.feature.zero_crossing_rate(y) #среднее частота пересечения нуля звукового временного ряда

    #Добавляем все параметры в один список
    out = [] # создаем пустой список
    for f in [np.min, np.mean, np.median]: out.append(f(rmse)) # добавляем среднеквадратичную амплитуду
    for f in [np.min, np.max]: out.append(f(spec_cent)) # добавляем спектральный центроид
    for f in [np.min]: out.append(f(spec_bw)) # добавляем ширину полосы частот
    for f in [np.min]: out.append(f(rolloff)) # добавляем спектральный спад частоты
    for f in [np.min]: out.append(f(zcr)) # добавляем пересечение нуля
    #добавляем среднее всех Частот цветности (по умолчанию 12 баков цветности)
    for f in [np.min]: out.append(f(list(map(f, librosa.feature.chroma_stft(y=y, sr=sr)))))
    #Возвращаем получившийся список размерностью (37,)
    return out

# Новая функция вывода результата на экран для поиска закономерностей по классам
def features_info(index_list):
    print('Таблица распределений', index_list)
    index_list = data.query(index_list).index
    lst = []
    for i in index_list:
        y, sr = librosa.load(data.loc[i,'original_path'])
        lst.append(get_features(y, sr))
    lst = pd.DataFrame(lst)
    display(lst.describe())
    print()

features_info('cls == 0')
features_info('cls == 1')

,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.000408,0.110517,0.091825,564.337291,2486.202508,468.680150,845.716553,0.027230,0.005862
std,0.000705,0.107028,0.111565,221.111705,453.906648,174.767781,556.390299,0.013647,0.018003
min,0.000003,0.000288,0.000270,3.684752,1187.459659,17.462004,10.766602,0.000000,0.000040
25%,0.000007,0.033695,0.014795,414.339442,2193.464879,345.804163,406.439209,0.018555,0.000117
50%,0.000015,0.068756,0.047375,492.887775,2577.050169,387.013352,592.163086,0.022461,0.000284
75%,0.000491,0.158417,0.112213,747.616290,2849.566062,667.712339,1356.591797,0.037109,0.002597
max,0.003001,0.392343,0.390436,1259.035116,3162.646604,997.874801,2702.416992,0.069336,0.128443


,0,1,2,3,4,5,6,7,8
count,4.030000e+02,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,1.615577e-04,0.059316,0.027073,388.724973,2753.503496,359.414644,390.162405,0.017605,0.000231
std,2.506953e-04,0.042651,0.040819,175.521036,318.150075,114.153666,217.747088,0.010845,0.002728
min,3.113112e-07,0.004644,0.000252,3.685809,1146.952381,17.674359,10.766602,0.000000,0.000004
25%,1.357068e-06,0.033359,0.004631,306.362699,2598.871860,343.522050,312.231445,0.010742,0.000010
50%,3.474475e-05,0.048702,0.016248,429.238713,2806.106322,382.604925,462.963867,0.018066,0.000014
75%,2.831662e-04,0.074189,0.034177,513.955759,2968.765179,407.729861,484.497070,0.024414,0.000023
max,2.526199e-03,0.407895,0.471528,1153.743349,3372.312191,768.584333,2164.086914,0.054199,0.054367


CPU times: total: 5min 39s
Wall time: 4min 9s


In [ ]:
#@title Разница между классами в признаках
def _features_info(index_list):
    lst = []
    for i in index_list:
        y, sr = librosa.load(data.loc[i,'original_path'])
        lst.append(get_features(y, sr))
    lst = pd.DataFrame(lst)
    display(lst.describe())

x0 = _features_info(index_list=data.query('cls == 0').index).loc['mean']
x1 = _features_info(index_list=data.query('cls == 1').index).loc['mean']
pd.DataFrame([x0, x1]).reset_index(drop=True)


,0,1,2,3,4,5,6,7,8
0,0.000408,0.110517,0.091825,564.337291,2486.202508,468.680150,845.716553,0.027230,0.005862
1,0.000162,0.059316,0.027073,388.724973,2753.503496,359.414644,390.162405,0.017605,0.000231


В таблице выше строки - классы. 0 - Ложный вызов, 1 - Авария.

Мы нашли разницу между классами в признаках для последующий кластеризации.

В таблице ниже подчеркнем разницу между классами. Равно удаленный промежуток между классами.

In [ ]:
#@title Равно удаленный промежуток между классами.
pd.DataFrame([pd.DataFrame([x0.loc['mean'], x1.loc['mean']]).mean(axis=0)])

,0,1,2,3,4,5,6,7,8
0,0.000285,0.084916,0.059449,476.531132,2619.853002,414.047397,617.939479,0.022417,0.003046


# Смещение промежуточного значения между классами. Бинарная балансировка.
Для начала конвертируем разницу между классами к общему - бинарному виду. Таким образом, мы увидим какие признаки лучше выделяют разницу между классами.

In [55]:
#@title Функция параметризации с бинарным выходом и определением смещения промежута
def get_features(y, sr, k):
    #Получаем различные параметры аудио
    rmse = librosa.feature.rms(y=y) # Среднеквадратичная амплитуда
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr) #среднее спектральныго центроида
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr) #среднее ширины полосы частот
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr) #среднее спектрального спада частоты
    zcr = librosa.feature.zero_crossing_rate(y) #среднее частота пересечения нуля звукового временного ряда
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr) # Частота цветности (по умолчанию 12 баков цветности)

    # конвертация значения в бинарный вариант по среднему значению и смещению k
    set_binary = lambda x, mean, k=1.0: 1 if x < mean*k else 0
    r_set_binary = lambda x, mean, k=1.0: 1 if x > mean*k else 0

    #Добавляем все параметры в один список
    out = [] # создаем пустой список
    
    # добавляем среднеквадратичную амплитуду
    out.append(set_binary(np.min(rmse), 0.000285, k))
    out.append(set_binary(np.mean(rmse), 0.084916, k))
    out.append(set_binary(np.median(rmse), 0.059449, k))
    
    # добавляем спектральный центроид
    out.append(set_binary(np.min(spec_cent), 476.531132, k))
    out.append(set_binary(np.max(spec_cent), 2619.853002 , k))
    
    # добавляем ширину полосы частот
    out.append(set_binary(np.min(spec_bw), 414.047397, k))
    
    # добавляем спектральный спад частоты
    out.append(set_binary(np.min(rolloff), 617.939479, k))
    
    # добавляем пересечение нуля
    out.append(set_binary(np.min(zcr), 0.022417 , k))
    
    #добавляем среднее всех Частот цветности (по умолчанию 12 баков цветности)
    chroma_stft = np.min(list(map(np.min, chroma_stft)))
    out.append(set_binary(chroma_stft, 0.003046, k))
    
    #Возвращаем получившийся список размерностью (9,)
    return out

# Новая функция вывода результата на экран для поиска закономерностей по классам
def features_info(index_list, k=1):
    print('Таблица распределений', index_list)
    index_list = data.query(index_list).index
    lst = []
    for i in index_list:
        y, sr = librosa.load(data.loc[i,'original_path'])
        lst.append(get_features(y, sr, k))
    lst = pd.DataFrame(lst)
    display(lst.describe())
    print()

In [46]:
%%time
#@title Бинарные значения без смещения промежутка между классами.
features_info('cls == 0', 1)
features_info('cls == 1', 1)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.650000,0.533333,0.583333,0.466667,0.516667,0.550000,0.533333,0.500000,0.750000
std,0.480995,0.503098,0.497167,0.503098,0.503939,0.501692,0.503098,0.504219,0.436667
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000
50%,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.500000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.749380,0.833747,0.908189,0.652605,0.265509,0.789082,0.972705,0.645161,0.992556
std,0.433909,0.372770,0.289118,0.476734,0.442153,0.408467,0.163145,0.479059,0.086065
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



CPU times: total: 6min 56s
Wall time: 3min 52s


Исходя из этих двух таблиц мы видим, что нули (Ложные вызовы) часто определяются как единицы (Аварии). Ложные вызовы определяются верно, в лучшем случае, на 50%. Бинарный метод без смещения относит большинтсво в "Аварии".

Значение равно удаленного промежутка между классами не идельно - классы имеют разный диапозон значений. "Ложные вызовы" попадают в категорию единиц, а "Аварии" в нули. Нужно сформировать такое промежуток, которое будет больше соответствовать диапозону значений этих двух групп. Для этого создадим смещение и найдем его лучшее значние для каждого признака эмпирическим путем.

In [57]:
%%time
#@title Смещение разделительной границы на 75% в пользу класса 1. Класс Аварии
features_info('cls == 0', 1.75)
features_info('cls == 1', 1.75)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.0,60.000000,60.000000,60.000000,60.000000
mean,0.766667,0.733333,0.700000,0.916667,1.0,0.983333,0.650000,0.783333,0.800000
std,0.426522,0.445948,0.462125,0.278718,0.0,0.129099,0.480995,0.415450,0.403376
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,1.0,1.000000,0.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.0,403.000000,403.000000,403.000000,403.000000
mean,0.955335,0.965261,0.957816,0.990074,1.0,0.997519,0.985112,0.975186,0.997519
std,0.206824,0.183347,0.201258,0.099255,0.0,0.049814,0.121257,0.155751,0.049814
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000



CPU times: total: 6min 32s
Wall time: 3min 54s


In [64]:
%%time
#@title Смещение разделительной границы на 50% в пользу класса 1. Класс Аварии
features_info('cls == 0', 1.5)
features_info('cls == 1', 1.5)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.0,60.000000,60.000000,60.000000,60.000000
mean,0.700000,0.716667,0.683333,0.666667,1.0,0.700000,0.633333,0.700000,0.766667
std,0.462125,0.454420,0.469102,0.475383,0.0,0.462125,0.485961,0.462125,0.426522
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.0,403.000000,403.000000,403.000000,403.000000
mean,0.935484,0.947891,0.950372,0.990074,1.0,0.987593,0.985112,0.923077,0.997519
std,0.245975,0.222523,0.217445,0.099255,0.0,0.110831,0.121257,0.266801,0.049814
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000



CPU times: total: 6min 34s
Wall time: 3min 54s


In [65]:
%%time
#@title Смещение разделительной границы на 25% в пользу класса 1. Класс Аварии
features_info('cls == 0', 1.25)
features_info('cls == 1', 1.25)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.0,60.000000,60.000000,60.000000,60.000000
mean,0.666667,0.633333,0.650000,0.616667,1.0,0.683333,0.633333,0.633333,0.766667
std,0.475383,0.485961,0.480995,0.490301,0.0,0.469102,0.485961,0.485961,0.426522
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.846154,0.910670,0.940447,0.972705,0.995037,0.972705,0.977667,0.853598,0.997519
std,0.361250,0.285574,0.236952,0.163145,0.070359,0.163145,0.147946,0.353948,0.049814
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



CPU times: total: 6min 16s
Wall time: 3min 55s


In [66]:
%%time
#@title Смещение разделительной границы на 25% в пользу класса 0. Класс Ложного вызова
features_info('cls == 0', .75)
features_info('cls == 1', .75)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.633333,0.483333,0.483333,0.133333,0.166667,0.066667,0.366667,0.166667,0.716667
std,0.485961,0.503939,0.503939,0.342803,0.375823,0.251549,0.485961,0.375823,0.454420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.652605,0.647643,0.821340,0.322581,0.022333,0.124069,0.526055,0.441687,0.987593
std,0.476734,0.478298,0.383544,0.468045,0.147946,0.330071,0.499941,0.497205,0.110831
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



CPU times: total: 6min 34s
Wall time: 3min 53s


In [67]:
%%time
#@title Смещение разделительной границы на 50% в пользу класса 0. Класс Ложного вызова
features_info('cls == 0', .5)
features_info('cls == 1', .5)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.616667,0.350000,0.366667,0.016667,0.016667,0.016667,0.066667,0.066667,0.683333
std,0.490301,0.480995,0.485961,0.129099,0.129099,0.129099,0.251549,0.251549,0.469102
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.585608,0.411911,0.692308,0.173697,0.007444,0.084367,0.243176,0.265509,0.980149
std,0.493229,0.492791,0.462112,0.379320,0.086065,0.278283,0.429534,0.442153,0.139662
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



CPU times: total: 6min 22s
Wall time: 3min 52s


In [68]:
%%time
#@title Смещение разделительной границы на 75% в пользу класса 0. Класс Ложного вызова
features_info('cls == 0', .25)
features_info('cls == 1', .25)

Таблица распределений cls == 0


,0,1,2,3,4,5,6,7,8
count,60.000000,60.000000,60.000000,60.000000,60.0,60.000000,60.000000,60.000000,60.000000
mean,0.600000,0.133333,0.250000,0.016667,0.0,0.016667,0.016667,0.033333,0.666667
std,0.494032,0.342803,0.436667,0.129099,0.0,0.129099,0.129099,0.181020,0.475383
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2,3,4,5,6,7,8
count,403.000000,403.000000,403.000000,403.000000,403.0,403.000000,403.000000,403.000000,403.000000
mean,0.543424,0.086849,0.478908,0.111663,0.0,0.076923,0.156328,0.173697,0.967742
std,0.498730,0.281963,0.500176,0.315342,0.0,0.266801,0.363617,0.379320,0.176904
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
50%,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000



CPU times: total: 5min 50s
Wall time: 3min 53s


Все прозведенные эксперименты не перечислены.

Теперь можно отфильтровать слабые признаки.

# Итоговый набор признаков

In [74]:
%%time
#@title Новая функция параметризации аудио
def get_features(y, sr, binary_out=False):

    #Получаем различные параметры аудио
    rmse = librosa.feature.rms(y=y) # Среднеквадратичная амплитуда
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr) # Cреднее спектрального спада частоты
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr) # Частота цветности (по умолчанию 12 баков цветности)

    # конвертация значения в бинарный вариант по среднему значению и смещению k
    set_binary = lambda x, mean, k=1.0: 1 if x < mean*k else 0
    r_set_binary = lambda x, mean, k=1.0: 1 if x > mean*k else 0

    #Добавляем все параметры в один список
    out = [] # создаем пустой список
    # добавляем среднеквадратичную амплитуду
    # 0
    if binary_out: out.append(set_binary(np.median(rmse), 0.059449, .5))
    else: out.append(np.median(rmse))
    # добавляем спектральный спад частоты
    # 1
    if binary_out: out.append(set_binary(np.min(rolloff), 617.939479, .8))
    else: out.append(np.min(rolloff))
    # добавляем среднее всех Частот цветности
    # 2
    chroma_stft = np.min(list(map(np.min, chroma_stft)))
    if binary_out: out.append(set_binary(chroma_stft, 0.003046, .02))
    else: out.append(chroma_stft)
    # возвращаем получившийся список размерностью (3,)
    return out

# Новая функция вывода результата на экран для поиска закономерностей по классам
def features_info(index_list, binary_out=False):
    print('Таблица распределений', index_list)
    index_list = data.query(index_list).index
    lst = []
    for i in index_list:
        y, sr = librosa.load(data.loc[i,'original_path'])
        lst.append(get_features(y, sr, binary_out=binary_out))
    lst = pd.DataFrame(lst)
    display(lst.describe())
    print()

features_info('cls == 0', binary_out=True)
features_info('cls == 1', binary_out=True)

Таблица распределений cls == 0


,0,1,2
count,60.000000,60.000000,60.000000
mean,0.366667,0.383333,0.083333
std,0.485961,0.490301,0.278718
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000



Таблица распределений cls == 1


,0,1,2
count,403.000000,403.000000,403.000000
mean,0.692308,0.918114,0.863524
std,0.462112,0.274532,0.343720
min,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000



CPU times: total: 4min 16s
Wall time: 2min 6s


Как мы видим, только цветность частот (Признак - 2) имеет верное для групп рапределение 25%, 50%, 75%. В 0 и 1 признаках часто попадают ложные результаты.

# Простая модель кластеризации на основе цветности частот - лучшего признака

Создадим простую бинарную модель на основе одного признака

In [4]:
easy_model = lambda y, sr: 1 if np.min(list(map(np.min, librosa.feature.chroma_stft(y=y, sr=sr)))) < 0.003046*.02 else 0
# Пример
y,sr = librosa.load('calls\\false\\false (2).mp3')
if easy_model(y,sr) == 0: print('Ложный вызов или false.')
else: print('Авария или true.')


Ложный вызов или false.


Добавим Predict модели их сигналам в таблицу и посчитаем сколько совпадет.

In [6]:
#@title Таблица аудио файлов с распознаниями модели (predict)
data = pd.read_csv('revshift_20.csv')
def predict(original_path, cls):
    y, sr = librosa.load(original_path)
    return easy_model(y,sr) == cls
lst=[]
for i in range(len(data)):
    lst.append(predict(data.loc[i,'original_path'], data.loc[i,'cls']))
data['predict'] = lst
display(data)


,label,cls,id,sample,original_duration,segments_20sec,segment_type,segment_number,duration,start_sec,stop_sec,original_path,comment,path,count,original_dir_cls,predict
0,false (2),0,90,train,19,0,full_audio,0,19,0,19,calls\false\false (2).mp3,augs-- gen repeat stft,dataset\train\false\false (2) id-90.mp3,23,false,True
1,false (2),0,94,train,19,0,full_audio,0,19,0,19,calls\false\false (2).mp3,augs-- gen repeat mx2y,dataset\train\false\false (2) id-94.mp3,23,false,True
2,false (2),0,98,train,19,0,full_audio,0,19,0,19,calls\false\false (2).mp3,augs-- gen repeat high_hz,dataset\train\false\false (2) id-98.mp3,23,false,True
3,false (2),0,102,train,19,0,full_audio,0,19,0,19,calls\false\false (2).mp3,augs-- gen repeat mn2y,dataset\train\false\false (2) id-102.mp3,23,false,True
4,false (2),0,106,train,19,0,full_audio,0,19,0,19,calls\false\false (2).mp3,augs-- gen repeat pink_noise,dataset\train\false\false (2) id-106.mp3,23,false,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2482,true (1),1,6840,test,53,4,segment,0,20,0,20,calls\true\true (1).mp3,augs--,dataset\test\true\true (1) id-6840.mp3,1,true,True
2483,true (23),1,6876,test,33,2,segment,0,20,0,20,calls\true\true (23).mp3,augs--,dataset\test\true\true (23) id-6876.mp3,1,true,True
2484,true (27),1,6892,test,56,4,segment,0,20,0,20,calls\true\true (27).mp3,augs--,dataset\test\true\true (27) id-6892.mp3,1,true,True
2485,true (36),1,6926,test,58,4,segment,0,20,0,20,calls\true\true (36).mp3,augs--,dataset\test\true\true (36) id-6926.mp3,1,true,True


In [7]:
#title True - верные ответы. False - ошибочные распознания
display(data.value_counts('predict'))


predict
True     2153
False     334
Name: count, dtype: int64

In [8]:
print('Число верных ответов: ',round(2153/(334+2153), 2))

Число верных ответов:  0.87


In [16]:
# Группировка ошибок
data.query('predict == False').value_counts('original_dir_cls')

original_dir_cls
Ложь_разговоры              173
Истина_посторонние_звуки     45
Истина_тишина                43
Истина_разговоры             42
true                         30
Ложь_посторонние_звуки        1
Name: count, dtype: int64